# Homework 2: Convolutional Neural Networks (100 points)

### Overview

With new knowledge of convolutional neural networks, we can model a more difficult image recognition task. The CIFAR-10 classification dataset consists of 60,000 labelled images split between 10 classes: airplanes, cars, birds, cats, deer, dogs, frogs, horses, ships, and trucks.

For the purposes of this assignment, we will compare two models on the same dataset: a fully connected neural network (as in Homework 1) called ANN and a new convolutional architecture called CNN, as outlined in the next section. To be fair, we attempt to allow the same number of trainable parameters in the ANN as the CNN, which means we need to use the same input transformation to flattened grayscale used in Homework 1 for the ANN. The CNN reaps the full benefit of the original 2D image in RGB.

### CNN Architecture

Each image consists of 32x32 RGB pixel values between 0 and 255. We do not need to perform any preprocessing as the convolutional model will use all three channels as concurrently as input.

The architecture in use has 5 layers: a convolution layer followed by a pooling layer, then another convolutional layer, then two fully connected dense layers. The latter of these has 10 neurons to provide classification output.

### Your Task

At the bottom of this notebook file, there are four short answer questions testing your understanding of this neural network architecture. As before, some questions will require you to experiment with model hyperparameters.

Below each question is a cell with the text “Type Markdown and LaTex.” Double-click the cell and type your response to the question. Save your responses by clicking on the floppy disk icon or choosing File - Save and Checkpoint.

After responding to the questions, download your notebook as a .PDF file by choosing File - Download as - pdf (.pdf). You will be submitting this .pdf to your instructor for grading.

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

In [3]:
torch.set_num_threads(4)
torch.set_num_interop_threads(4)

In [4]:
trainTransform = transforms.Compose([transforms.RandomRotation(5),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])
testTransform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                    ])

In [5]:
root_dir = 'assets_week2'
trainDataset = torchvision.datasets.CIFAR10(root=root_dir, train=True, download=True, transform=trainTransform)
trainLoader = torch.utils.data.DataLoader(trainDataset, batch_size=4, shuffle=True, num_workers=2)
testDataset = torchvision.datasets.CIFAR10(root=root_dir, train=False, download=True, transform=testTransform)
testLoader = torch.utils.data.DataLoader(testDataset, batch_size=4, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class ANNModel(nn.Module):
    def __init__(self, hiddenSize, dropoutRate, activate):
        super().__init__()
        # Note that 'layer' and 'dense' differ only in name (to show similarity to CNN)
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.layer1 = nn.Linear(1024, 100)
        self.layer2 = nn.Linear(100, 15 * 5 * 5)
        self.dense1 = nn.Linear(15 * 5 * 5, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, 10)
        
    def forward(self, x):
        x = self.activate(self.layer1(x))
        x = self.activate(self.layer2(x))
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

class CNNModel(nn.Module):
    def __init__(self, hiddenSize, outChannels, dropoutRate, activate):
        super().__init__()
        self.outChannels = outChannels
        self.activate = nn.Sigmoid() if activate == "Sigmoid" else nn.ReLU()
        self.conv1 = nn.Conv2d(3, 24, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(24, outChannels, 5)
        self.dense1 = nn.Linear(outChannels * 5 * 5, hiddenSize)
        self.dropout = nn.Dropout(dropoutRate)
        self.dense2 = nn.Linear(hiddenSize, 10)

    def forward(self, x):
        x = self.pool(self.activate(self.conv1(x)))
        x = self.pool(self.activate(self.conv2(x)))
        x = x.view(-1, self.outChannels * 5 * 5)
        x = self.dropout(self.activate(self.dense1(x)))
        return self.dense2(x)

In [7]:
# Number of neurons in the first fully-connected layer
hiddenSize = 100
# Number of feature filters in second convolutional layer
numFilters = 25
# Dropout rate
dropoutRate = 0
# Activation function
activation = "ReLU"
# Learning rate
learningRate = 0.001
# Momentum for SGD optimizer
momentum = 0.9
# Number of training epochs
numEpochs = 10

In [8]:
ann = ANNModel(hiddenSize, dropoutRate, activation)
cnn = CNNModel(hiddenSize, numFilters, dropoutRate, activation)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(list(ann.parameters()) + list(cnn.parameters()), lr=learningRate, momentum=momentum)

print('>>> Beginning training!') 
ann.train()
cnn.train()
for epoch in range(numEpochs):  # loop over the dataset multiple times
    annRunningLoss, cnnRunningLoss = 0, 0
    for i, (inputs, labels) in enumerate(trainLoader, 0):
        annInputs = torch.sum(inputs, axis=1).view(-1, 32*32)
        
        optimizer.zero_grad()

        # Forward propagation
        annOutputs = ann(annInputs)
        cnnOutputs = cnn(inputs)
        
        # Backpropagation
        annLoss = criterion(annOutputs, labels)
        cnnLoss = criterion(cnnOutputs, labels)
        annLoss.backward()
        cnnLoss.backward()
        
        # Gradient update
        optimizer.step()

        annRunningLoss += annLoss.item()
        cnnRunningLoss += cnnLoss.item()
        if (i+1) % 2000 == 0:    # print every 2000 mini-batches
            print('Epoch [{}/{}], Step [{}/{}], ANN Loss: {}, CNN Loss: {}'.format(epoch + 1, numEpochs, i + 1, len(trainDataset)//4, annRunningLoss/2000, cnnRunningLoss/2000))
            annRunningLoss, cnnRunningLoss = 0, 0

print()
print('>>> Beginning validation!')
ann.eval()
cnn.eval()
annCorrect, cnnCorrect = 0, 0
total = 0
for inputs, labels in testLoader:
    annInputs = torch.sum(inputs, axis=1).view(-1, 32*32)
    annOutputs = ann(annInputs)
    cnnOutputs = cnn(inputs)
    _, annPredicted = torch.max(annOutputs.data, 1)
    _, cnnPredicted = torch.max(cnnOutputs.data, 1)
    total += labels.size(0)
    annCorrect += (annPredicted == labels).sum().item()
    cnnCorrect += (cnnPredicted == labels).sum().item()
print('ANN validation accuracy: {}%, CNN validation accuracy: {}%'.format(annCorrect / total * 100, cnnCorrect / total * 100))

>>> Beginning training!
Epoch [1/10], Step [2000/12500], ANN Loss: 2.085167105138302, CNN Loss: 2.0130053512156008
Epoch [1/10], Step [4000/12500], ANN Loss: 1.9335808462500572, CNN Loss: 1.6616046900749206
Epoch [1/10], Step [6000/12500], ANN Loss: 1.8763703825175762, CNN Loss: 1.498439008742571
Epoch [1/10], Step [8000/12500], ANN Loss: 1.8333953123986722, CNN Loss: 1.426016979753971
Epoch [1/10], Step [10000/12500], ANN Loss: 1.8128380991816522, CNN Loss: 1.3771966981217265
Epoch [1/10], Step [12000/12500], ANN Loss: 1.7761154870539904, CNN Loss: 1.32719108036533
Epoch [2/10], Step [2000/12500], ANN Loss: 1.731828540638089, CNN Loss: 1.2460496757104993
Epoch [2/10], Step [4000/12500], ANN Loss: 1.7247903662919999, CNN Loss: 1.2009598610028624
Epoch [2/10], Step [6000/12500], ANN Loss: 1.7200220656841994, CNN Loss: 1.1878086058944464
Epoch [2/10], Step [8000/12500], ANN Loss: 1.7040260971039534, CNN Loss: 1.1459970939401538
Epoch [2/10], Step [10000/12500], ANN Loss: 1.69641349837183

## Homework Questions

### Question 1: CNN Advantage (10 points)

Compute the accuracy of a simple dense neural network and a simple CNN on the dataset. Explain the results and briefly overview the advantages of a CNN over a standard neural network for image-related tasks.

The validation accuracy for the simple ANN, utilizing the above code with the default parameters, is 43.169%, while the validation accuracy for the CNN is 66.59%. At each step during the training process, the loss minimization is greater for the CNN than for the ANN, even near the end when diminishing returns starts to set in. 

While ANN is a good general starting point for image classification related tasks, there are a few disadvantages it has that may influence someone to choose a CNN architecture instead. One disadvantage is that, prior to piping the image into the input layer of the ANN, it is necessary to convert the image into a one dimensional vector. This poses a few problems, one of them being that converting the image to a vector in this manner can greatly increase the number of trainable parameters to the tune of hundreds of thousands or more. Another disadvantage of this conversion  is that the valuable signal from the spatial features of the image is also lost. Additionally, ANN can also suffer from vanishing/exploding gradient during backpropogation depending on the activation function used, which can be exacerbated if the network is very deep(many hidden layers).


CNN on the other hand performs much better on image classification related tasks. The key buildings blocks of the CNN are the convolutional layers, which have filters/kernels that extract the relevant spatial features from the input image. These filters are learned by the CNN automatically during training. This is a big advantage for CNN over ANN, because spatial features - in other words the arrangement of the pixels in an image - can be critical for identifying the image properly. For instance, if the input image is a human face, it can be easy to identify the image as such using a CNN because it can learn the location of important facial features in relation to each other, such as nose, mouth, ears, etc. This will help it to differentiate it from the facial features of something else, like a dog or a cat, which an ANN might not be able to do very well because it doesn't learn the valuable info from spatial features. Furthermore, even though CNN is still susceptible to exploding/vanishing gradient, because it learns fewer parameters than ANN, the problem is less prevalent. 



### Question 2: Dropout Rate (25 points)

Explain the purpose of dropout in any neural network model. In doing so, note what can happen if the dropout rate is too high and what can happen if the dropout rate is too low.

Because of the power of neural network models, if the training data fed to it is lacking in number of training points, it is very easy for it to overfit the training data. Although it is possible to use ensemble methods and "stack" multiple neural network models together as a way to try and circumvent the problem of overfitting, this may become cumbersome because it will inevitably result in greater computational cost and time to train. Another solution, therefore, would be to randomly "drop out" nodes during the training process, which is not only much less computationally expensive, but is also as good of a way, if not better, than other methods to prevent overfitting and reduce the generalization error. 

Dropout can be used in both fully connected layers and convolutional layers. During each iteration, some % of nodes are randomly ignored, meaning all incoming and outgoing connections to that node are dropped, in effect introducing deliberate noise into the training process, which randomly forces different nodes at each iteration to take on more or less responsibility for the inputs, which in turn reduces dependency on any one specific connection. This also helps to combat the vanishing gradient issue that can occasionally occur when there are a very large number of nodes. 


Of course, by introducing dropout into the model, it will of course also cause the capacity of the model to drop. If the dropout rate is too high, this may cause the model to be unable to fit the training data properly, which will inevitably lead to bad accuracy. A higher droput rate may result in too high variance with some of the layers, which will degrade the training process and prevent the model from learning enough information to effectively generalize. On the other hand, if the dropout rate is too low, this will have an insignificant impact in regards to the accuracy and reduction of the generalization error. 







### Question 3: Kernel Size (25 points)

Explain the purpose of spatial filters (kernels) in a CNN. Additionally, explain where they fit into the overall architecture of the CNN in this coding example. Finally, explain what can happen if the kernel size is too large and what can happen if the kernel size is too small.

CNN's can be composed of several different kinds of layers, such as convolutional layers, pooling layers, nonlinearity layers, and fully connected layers. The key building block of CNN's are the convolutional layers. Within the convolutional layers, the spatial filter(or kernel) is applied to the input. These filters are learned during the training process, and repeated application of the filters to an input results in the creation of a feature map(also called an activation map) that essentially displays the location and the strength of a detected feature in the input image(in the context of image-related tasks). Although filters can be "pre-created" by the data scientist, CNNs are advantgeous in the sense that it is not crucially necessary to do that since the filters can be learned automatically by the CNN.

The filter can be visualized as a two dimensional array of weights, and is supposed to be smaller than the original input image. Within the convolutional layer, the filter, along with an equivalent sized patch of data from the input, are multiplied together using the dot product, resulting in a single value. This filter is then systematically shifted across the input data, computing the dot product each time, until it has been passed across all of the input data. The end result from this is a set of values called an activation map, or feature map. This approach of moving the spatial filter across the input data is a good one, because it implies that, no matter where a specific feature is located in the input data, the filter will have an opportunity to detect it. Upon creating the feature map, the feature map is then often passed to a nonlinearity layer like ReLU. It is worth mentioning that CNN's don't just learn one filter but learn many in parallel in order to learn many different ways of "seeing" the input data. Furthermore, it is also possible to stack the convolution layers together - there doesn't have to be just one. This is advantageous in the sense that, by doing this, a convolutional layer can take the output of a preceding one and learn features that are "combinations" of other features.

In the context of the code above, there are two convolutional layers, and they are stacked on top of each other. Furthermore, the non-linearity that is applied to the output from them is specified as "Sigmoid". The output from the convolutional layers are also passed through pooling layers. When an image is passed through the pooling layer, it essentially reduces the size of the image by performing aggregation over it(using max pooling or average pooling). The end result is an image that is, of course, smaller, while also intensifiying useful features in the image. The convolutional layers and the pooling layers work together. The convolutional layers extracts the important features in an image, and the pooling layers intensify the important features discovered by the convolutional layers, while also discarding info that is not relevant.

A potential issue with using a kernel size that is too small - as an extreme example lets say 1X1 - is that the features extracted from the filter may be too fine-grained and thus not contain potentially useful information from neighboring pixels. If the kernel is excessively large, however, then the filter will essential only learn global features and may miss important local ones. Furthermore, useful features may be found in more than once place within an image, so it is advantageous to slide the filter along the image as much as possible to extract these. Finally, larger kernels would increase the computational cost of training(more weights = more calculations, and thus more time to train).







### Question 4: Data Augmentation (40 points)

Use the code snippet provided in the next box to implement data augmentation by updating the contents of box 2 and re-running the model. Compare your accuracy without and with data augmentation and explain the results. In doing so, explain the purpose of data augmentation.

In [1]:
# transforms.RandomRotation(5)

Without data augmentation, using the default parameters,  the accuracy given for the ANN model is 42.57% and the accuracy given for the CNN model is 68.27%. Total time to train was 26 minutes. With the above rotation data augmentation included in the code, and with other parameters remaining as default, the accuracy given for the ANN model is 43.83% and the accuracy given for the CNN model is 69.47. This is marginally better accuracy for both models. Additionally, time to train was a bit longer, at 27 minutes. 

I somewhat anticipated behavior indicating improvement would occur upon including the above snippet within the model code. It is often the case that simply feeding more training data is a reasonable method for improving the accuracy of ones model. If the model has more examples to work with during training, then naturally, it will be able to generalize better. Data augmentation is one such method of providing the model with more examples to work with during training without explicitly gathering more "real" training observations. In other words, "fake" data is added to the training data by transforming the input data through operations such as rotation, cropping, mirroring, jittering, flipping, and more. The purpose behind these transformations is to provide the model with variations of the input data that is plausible for it to see when it is time to test, while retaining its output classification label(a label of dog will still be dog even if it is transformed via data augmentation). For example, if the input data is an image of a dog, it may make sense to flip the image horizontally as a form of augmentation, since it is quite likely for a dog to be facing different directions in any given photo. This will allow the model to not be completely thrown off when it receives test images that have these slight alterations in them. However, it is also important to make sure that the data augmentation one is performing is logical. For instance, in the case of an image of a dog, it may not make a lot of intuitive sense to flip the image vertically as a form of augmentation since it is very unlikely for a dog to be standing "upside down." Other logical operations one could do would be to shift the image - in other words, clip part of the dogs anatomy out of the image - zoom in on the image, or change the brightness of the image.



